## Notebook for downloading inputs to create SLIIDERS-ECON

This notebook contains directions for downloading various input datasets to create the final product for this directory, the **SLIIDERS-ECON** dataset.

In general, we will keep the format, file name, and data unaltered, but apply changes when
- file name is not human-readable, too long, or is not much informative about the dataset (assign appropriate file names)
- file format causes errors (save in a similar file format that is not error-prone)

In [ ]:
import os
import ssl
import subprocess
import tarfile
from pathlib import Path
from urllib import request as urequest
import zipfile

import dask.distributed as dd
import pandas as pd
from dask_gateway import Gateway
from pandas_datareader import wb as dr_wb

from sliiders import settings as sset

# dask gateway setup
gateway = Gateway()
image_name = sset.DASK_IMAGE

In [ ]:
# creating select directories
PWT_DIRECTORY = sset.PATH_PWT_RAW.parent
IMF_WEO_DIRECTORY = sset.PATH_IMF_WEO_RAW.parent
MPD_DIRECTORY = sset.PATH_MPD_RAW.parent
GWDB_DIRECTORY = sset.PATH_GWDB2021_RAW.parent
SRTM15PLUS_DIRECTORY = sset.PATH_SRTM15_PLUS.parent
MDT_DIRECTORY = sset.PATH_GEOG_MDT_RAW.parent

directories_to_create = [
    PWT_DIRECTORY,
    IMF_WEO_DIRECTORY,
    MPD_DIRECTORY,
    GWDB_DIRECTORY,
    SRTM15PLUS_DIRECTORY,
    MDT_DIRECTORY,
    sset.DIR_WB_WDI_RAW,
    sset.DIR_LITPOP_RAW,
    sset.DIR_GEG15_RAW,
    sset.DIR_CCI_RAW,
    sset.DIR_UN_WPP_RAW,
    sset.DIR_UN_AMA_RAW,
    sset.DIR_ALAND_STATISTICS_RAW,
    sset.DIR_OECD_REGIONS_RAW,
    sset.DIR_LANDSCAN_RAW,
    sset.DIR_IIASA_PROJECTIONS,
    sset.DIR_GEOG_DATUMS_XGM2019e_WGS84,
    sset.DIR_GEOG_DATUMS_EGM96_WGS84,
    sset.DIR_GADM,
]
for direc in directories_to_create:
    direc.mkdir(exist_ok=True, parents=True)

## Fetching raw data from various sources

### Penn World Tables 10.0 (PWT 10.0)

In [ ]:
# PWT10.0
pwt100_data = pd.read_excel("https://www.rug.nl/ggdc/docs/pwt100.xlsx", sheet_name=2)

# PWT10.0 capital details
pwt100_data_K = pd.read_excel(
    "https://www.rug.nl/ggdc/docs/pwt100-capital-detail.xlsx", sheet_name=2
)

pwt_filenames = ["pwt_100.xlsx", "pwt_K_detail_100.xlsx"]
for i, data in enumerate([pwt100_data, pwt100_data_K]):
    data.to_excel(
        excel_writer=(PWT_DIRECTORY / pwt_filenames[i]),
        sheet_name="Sheet1",
        index=False,
    )

### Maddison Project Dataset (MPD, Maddison Project Database 2020)

In [ ]:
madd = pd.read_excel(
    "https://www.rug.nl/ggdc/historicaldevelopment/maddison/data/mpd2020.xlsx",
    sheet_name=2,
)
madd.to_excel(
    excel_writer=(sset.PATH_MPD_RAW),
    index=False,
    sheet_name="Sheet1",
)

### World Bank WDI (WB WDI)

#### Investment-to-GDP ratio, GDP and GDPpc (nominal and PPP), and Population

In [ ]:
# country name and iso3 country code information
country_info = dr_wb.get_countries()[["name", "iso3c"]].rename(
    columns={"name": "country", "iso3c": "ccode"}
)

# relevant indicator information for the `dr_wb` module to fetch the variables
wbwdi_indicators = [
    "SP.POP.TOTL",  # population
    "NE.GDI.FTOT.ZS",  # investment-to-GDP ratio
    "NY.GDP.MKTP.PP.KD",  # GDP PPP
    "NY.GDP.PCAP.PP.KD",  # GDP per capita PPP
    "NY.GDP.MKTP.KD",  # GDP nominal
    "NY.GDP.PCAP.KD",  # GDP per capita nominal
]

j = 0
for indi in wbwdi_indicators:
    indi_info = (
        dr_wb.download(indicator=indi, country="all", start=1950, end=2020)
        .reset_index()
        .astype({"year": "int64"})
        .merge(country_info, on=["country"], how="left")
        .set_index(["ccode", "year"])
    )

    if j == 0:
        j += 1
        wbwdi_info = indi_info.copy()
    else:
        wbwdi_info = wbwdi_info.merge(
            indi_info.drop(["country"], axis=1),
            left_index=True,
            right_index=True,
            how="outer",
        )

# excluding those that have no information and saving the data
wb_info_vars = [x for x in wbwdi_info.columns if x != "country"]
wbwdi_info = wbwdi_info.loc[~pd.isnull(wbwdi_info[wb_info_vars]).all(axis=1), :]
wbwdi_info.to_parquet(sset.DIR_WB_WDI_RAW / "wdi_pop_iy_gdp.parquet")

#### WB WDI: exchange rate

In [ ]:
# country name and iso3 country code information
country_info = dr_wb.get_countries()[["name", "iso3c"]].rename(
    columns={"name": "country", "iso3c": "ccode"}
)

xr_code = "PA.NUS.FCRF"
xr_wb = dr_wb.download(indicator=xr_code, country="all", start=1950, end=2019)
xr_wb = (
    xr_wb.reset_index()
    .astype({"year": "int64"})
    .merge(country_info, on=["country"], how="left")
)
(
    xr_wb.set_index(["ccode", "year"])
    .rename(columns={xr_code: "xrate"})
    .to_parquet(sset.DIR_WB_WDI_RAW / "wdi_xr.parquet")
)

### UN WPP populations (overall and by-population-group data)

In [ ]:
# overall information
un_df = pd.read_csv(
    "https://population.un.org/wpp/Download/Files/"
    "1_Indicators%20(Standard)/CSV_FILES/WPP2019_TotalPopulationBySex.csv"
)

# by_age_group
by_age = pd.read_csv(
    "https://population.un.org/wpp/Download/Files/1_Indicators"
    "%20(Standard)/CSV_FILES/WPP2019_PopulationByAgeSex_Medium.csv"
)

# exporting
un_df.to_csv(sset.DIR_UN_WPP_RAW / "UN_WPP2019_TotalPopulation.csv", index=False)
by_age.to_csv(sset.DIR_UN_WPP_RAW / "UN_WPP2019_Population_by_Age.csv", index=False)

### Åland Island GDP and population (from Statistics and Research Åland or ÅSUB)

Note when newer versions are available, old links from ÅSUB will become deprecated; the below links in `ALA_GDP_LINK` and `ALA_POP_LINK` are valid as of 2022-03-29.

In [ ]:
# links
ALA_GDP_LINK = (
    "https://www.asub.ax/sites/www.asub.ax/files/attachments/page/nr005en.xls"
)
ALA_POP_LINK = (
    "https://www.asub.ax/sites/www.asub.ax/files/attachments/page/alv01_aland_faroe"
    "_islands_and_greenland_-_an_overview_with_comparable_data.xlsx"
)

# datasets read-in
ala_gdp = pd.read_excel(ALA_GDP_LINK, header=3)
ala_pop = pd.read_excel(ALA_POP_LINK, header=2, sheet_name="Population development")

# exporting
ala_gdp.to_excel(sset.DIR_ALAND_STATISTICS_RAW / "aland_gdp.xlsx", index=False)
ala_pop.to_excel(sset.DIR_ALAND_STATISTICS_RAW / "aland_pop.xlsx", index=False)

### Global Wealth Databook (from Credit Suisse)

We download the 2021 vintage (latest as of 2022-03-21).

In [ ]:
URL_GWDB = (
    "https://www.credit-suisse.com/media/assets/corporate/docs/about-us/research"
    "/publications/global-wealth-databook-2021.pdf"
)

gwr_raw = urequest.urlopen(URL_GWDB)
file = open(str(sset.PATH_GWDB2021_RAW), "wb")
file.write(gwr_raw.read())
file.close()

### LitPop (Eberenz et al. 2020, Earth Syst. Sci. Data)

#### Download Data from the Internet

In [ ]:
# link for downloading the LitPop files
link_base = (
    "https://www.research-collection.ethz.ch/bitstream/handle/20.500.11850/331316"
)

# readme, data, normalized data, and metadata
links = [
    link_base + "/_readme_v1_2.txt?sequence=18&isAllowed=y",
    link_base + "/LitPop_v1_2.tar?sequence=16&isAllowed=y",
    link_base + "/Lit_Pop_norm_v1.tar?sequence=4&isAllowed=y",
    link_base + "/_metadata_countries_v1_2.csv?sequence=12&isAllowed=y",
]

In [ ]:
def litpop_download(link, direc=sset.DIR_LITPOP_RAW):
    """Given a URL link, downloads (LitPop-related) data from the web and saves it in
    the specified local directory. The file name is parsed so that anything after the
    string `?sequence` is dropped (e.g., `file.txt?sequence=..` to `file.txt`).

    Parameters
    ----------
    link : str
        URL link for the file online
    direc : str or pathlib.Path
        directory to store the LitPop datasets

    Returns
    -------
    None, but saves the file downloaded from online to `direc`.

    """
    if type(direc) is str:
        direc = Path(direc)

    stop = link.find("?sequence")
    start = link.rfind("/", 0, stop) + 1
    urequest.urlretrieve(link, direc / link[start:stop])

    return None

In [ ]:
# cluster setup
N_CLUSTER = len(links)
cluster = gateway.new_cluster(worker_image=image_name, profile="micro")
client = cluster.get_client()
cluster.scale(N_CLUSTER)
cluster

In [ ]:
# takes approximately 20 minutes
futures = client.map(litpop_download, links)
dd.progress(futures)

In [ ]:
cluster.scale(0)
client.close()
cluster.close()
cluster.shutdown()

#### Un-tar and clear storage

We only un-tar the regular (not normalized) LitPop data here.

In [ ]:
# un-tar
regular_litpop = sset.DIR_LITPOP_RAW / "LitPop_v1_2.tar"
with tarfile.open(regular_litpop) as file:
    file.extractall(sset.DIR_LITPOP_RAW)

# clear storage for the existing tar file
os.remove(regular_litpop)

### GEG-15

We download 2'30" GEG15 and unzip.

In [ ]:
# downloading
zip_url = (
    "https://data.humdata.org/dataset/1c9cf1eb-c20a-4a06-8309-9416464af746/"
    "resource/e321d56d-022e-4070-80ac-f7860646408d/download/gar-exp.zip"
)
zip_path = sset.DIR_GEG15_RAW / "gar-exp.zip"
urequest.urlretrieve(zip_url, zip_path)

# unzipping
outpath = sset.DIR_GEG15_RAW / zip_path.stem
os.makedirs(outpath, exist_ok=True)
subprocess.Popen(["unzip", f"{zip_path}", "-d", f"{outpath}"])

In [ ]:
# remove zip file (use after unzipping)
os.remove(zip_path)

### Country-level Construction Cost Index from [Lincke and Hinkel (2021, *Earth's Future*)](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2020EF001965?campaign=woletoc)

The accompanying GitHub repository to Lincke and Hinkel (2021) is at [this link](https://github.com/daniellincke/DIVA_paper_migration).

In [ ]:
with open(sset.PATH_EXPOSURE_LINCKE, "wb") as f:
    f.write(
        requests.get(
            "https://raw.githubusercontent.com/daniellincke/"
            "DIVA_paper_migration/master/data/csv/country_input.csv"
        ).content
    )

### SRTM 15+

We use version 2.3.

In [ ]:
# Workaround for urllib request error
ssl._create_default_https_context = ssl._create_unverified_context
URL_SRTM15 = "https://topex.ucsd.edu/pub/srtm15_plus/SRTM15_V2.3.nc"

urequest.urlretrieve(URL_SRTM15, SRTM15PLUS_DIRECTORY / URL_SRTM15.split("/")[-1])

### GADM v3.6

In [ ]:
URL_GADM = "https://biogeo.ucdavis.edu/data/gadm3.6/gadm36_levels_gpkg.zip"
PATH_GADM_ZIP = sset.DIR_GADM / URL_GADM.split("/")[-1]
urequest.urlretrieve(URL_GADM, PATH_GADM_ZIP)

sset.PATH_GADM.parent.mkdir(exist_ok=True)

with zipfile.ZipFile(PATH_GADM_ZIP, mode="r") as z:
    z.extractall(sset.PATH_GADM.parent)

## Further data requiring separate manual instructions

In all cases below, `sset` is defined by `from sliiders import settings as sset` as above.

### UN Statistics National Accounts (Analysis of Main Aggregates; abbreviated as UN AMA)

#### UN AMA nominal (current prices) GDP per capita information

1. Travel to this [link](https://unstats.un.org/unsd/snaama/Basic) to get to the UN Statistics National Accounts search page.
2. Select all countries and all years available, and select "GDP, Per Capita GDP - US Dollars".
3. Select "Export to CSV", and you will download the file `Results.csv`. Rename this file as `un_snaama_nom_gdppc.csv`. We save this in `sset.DIR_UN_AMA_RAW`.

#### UN AMA nominal (current prices) GDP information

1. Similar to the nominal GDP per capita information, travel to this [link](https://unstats.un.org/unsd/snaama/Basic) to get to the UN Statistics National Accounts search page.
2. Select all countries and all years available, and select "GDP, at current prices - US Dollars".
3. Select "Export to CSV", and you will download the file `Results.csv`. Rename this file as `un_snaama_nom_gdp.csv`. We save this in `sset.DIR_UN_AMA_RAW`.

### OECD region-level information

#### OECD: population (region-level)
1. Go to the following OECD Stat website: link [here](https://stats.oecd.org/)
2. On the left, find the header "Regions and Cities" and click the "+" button.
3. From the drop down menu, click on "Regional Statistics".
4. Again from the drop down menu, click on "Regional Demography."
5. Finally, select "Population by 5-year age groups, small regions TL3." Make sure that "Indicator" is selected as "Population, All ages".
6. Download the file by selecting "Export," then "Text File (CSV)."
7. When a pop-up appears, select "Default format" then "Download." Rename the file as `REGION_DEMOGR.csv` (due to it having random-ish numeric parts in the name). Note that this step may take a longer time than others.
8. Finally, move the said file to `sset.DIR_OECD_REGIONS_RAW`.

#### OECD: GDP (region-level, in millions of constant 2015 PPP USD)
1. Similar to the population information, go to the following OECD Stat website: link [here](https://stats.oecd.org/)
2. On the left, find the header "Regions and Cities" and click the "+" button.
3. From the drop down menu, click on "Regional Statistics".
4. Again from the drop down menu, click on "Regional Economy."
5. Finally, select "Gross Domestic Product, Small regions TL3." Make sure that "Measure" is selected as "Millions USD, constant prices, constant PPP, base year 2015".
6. Download the file by selecting "Export," then "Text File (CSV)."
7. When a pop-up appears, select "Default format" then "Download." Rename the file as `REGION_ECONOM.csv` (due to it having random-ish numeric parts in the name). Note that this step may take a longer time than others.
8. Finally, move the said file to `sset.DIR_OECD_REGIONS_RAW`.

### IMF investment-to-GDP ratio, population, and GDP

1. Travel to this [link](https://www.imf.org/en/Publications/SPROLLs/world-economic-outlook-databases#sort=%40imfdate%20descending) to get to the World Economic Outlook Databases page.
2. Click on the latest "World Economic Outlook Database" link on the page; for our purposes, we have used the latest available one, which was "World Economic Outlook Database, October 2021" (may be updated in the future).
3. Click "By Countries", then click "ALL COUNTRIES", then click "CONTINUE" on the page that says "Select Countries."
4. Under the "NATIONAL ACCOUNTS" tab, check the following categories:
   - Gross domestic product, current prices (U.S. DOLLARS)
   - Gross domestic product per capita, current prices (U.S. DOLLARS)
   - Gross domestic product per capita, constant prices (PURCHASING POWER PARITY; 2017 INTERNATIONAL DOLLARS)
   - Total investment (PERCENT OF GDP)
5. Under the "PEOPLE" tab, check the category "Population," then click on "CONTINUE."
6. Under the tab "DATE RANGE," use the earliest year for "Start Year" (1980, in our case), and the latest non-future year for "End Year" (2020, in our case).
7. Under the tab "ADVANCED SETTINGS", click on "ISO Alpha-3 Code" for getting country codes. 
8. Click on "PREPARE REPORT." Then, click on "DOWNLOAD REPORT." Saved data should be in Excel format and be named `WEO_Data.xls`.
9. Open the said file on Excel, and re-save it in a preferred format of choice (we chose `.xlsx`); this is because the original file formatting is incompatible with Python and causes the error `ValueError: Excel file format cannot be determined, you must specify an engine manually.`
10. In our implementation, we save this file as `sset.PATH_IMF_WEO_RAW`.

### World Bank Intercomparison Project 2017 (WB ICP 2017): Construction Cost Index

While most World Bank data can be downloaded by using `pandas_datareader.wb`, it seems that variables in WB ICP 2017 - including `1501200:CONSTRUCTION`, which is necessary for SLIIDERS-ECON - cannot be downloaded using the said module (despite being searchable in the module using `pandas_datareader.wb.search`). Therefore, we follow the below manual process for downloading the WB ICP 2017 dataset.
1. Use [this link](https://databank.worldbank.org/embed/ICP-2017-Cycle/id/4add74e?inf=n) to access WB ICP 2017 in table format.
2. Upon entering the webpage, look to the upper right corner and click on the icon with downward arrow with an underline. This should prompt the download.
3. When the download finishes, there should be a `.zip` file called `ICP 2017 Cycle.zip`. Access the `.csv` file whose name ends in `_Data.csv` (there should be two files in the `.zip` file, the other being a file whose name ends in `_Series - Metadata.csv`).
4. Save that `.csv` file as `sset.PATH_EXPOSURE_WB_ICP`.

### IIASA and OECD models' GDP and population projections (2010-2100, every 5 years)

1. Go to the following IIASA SSP Database website: link [here](https://tntcat.iiasa.ac.at/SspDb); you may need to register and create your log-in.
2. In the above tabs, there is a tab called "Download"; click on it.
3. Under "SSP Database Version 2 Downloads (2018)" and under the sub-header "Basic Elements", there is a download link for `SspDb_country_data_2013-06-12.csv.zip`. Click and download the said `.zip` file.
4. Extract and save the `SspDb_country_data_2013-06-12.csv`. Again, for our purposes, we save this in `sset.DIR_IIASA_PROJECTIONS`.

### LandScan 2019

1. To download this dataset, you need to first apply for an Oak Ridge National Laboratory account (link [here](https://landscan.ornl.gov/user/apply)).
2. After having gained access, go to the said website, click on "DOWNLOAD" -> "LandScan Datasets" -> "Continue to download" next to LandScan 2019.
3. Click on "By downloading LandScan 2019 I agree to the above terms" in the following webpage; this will download the file `LandScan Global 2019.zip`. We save this in `sset.DIR_LANDSCAN_RAW`.

### Global geoids, based on select Earth Gravitational Models (EGMs)
1. Go to the following International Centre for Global Earth Models (ICGEM) website (link [here](http://icgem.gfz-potsdam.de/calcgrid)) to reach the page "Calculation of Gravity Field Functionals on Ellipsoidal Grids".
2. Under **Model Selection**, select `XGM2019e_2159`.
3. Under **Functional Selection**, select `geoid`.
4. Under **Grid selection**, there's a **Grid Step [°]** option. Change the value to **0.05**. Also, make sure that the **Reference System** is `WGS84`.
5. Due to download size constraints, we need to download this data in 4 chunks. Do the following:
   - Split the full range of latitudes and longitudes in half, which yields the following 4 combinations of longitude-latitude ranges: $([-180, 0], [-90, 0]), ([-180, 0], [0, 90]), ([0, 180], [-90, 0])$, and $([0, 180], [0, 90])$.
   - Under **Grid selection** again, one can select the range of longitudes and latitudes. Select one of the above combinations and press `start computation`.
   - This will open up a new tab for calculations, which may take some time to complete. Once this is done, press **Download Grid**.
   - Once the download is complete, go back to the previous page with **Model selection**, **Functional selection**, and more. Make sure the selections you made are intact, select another longitude-latitude combination, and repeat the process until there are no combinations left.
6. Once the above steps are done, go back to Step 2 above; but instead of selecting `XGM2019e_2159` for **Model selection**, select `EGM96`. Go through the Steps 3 to 5 again with this new selection.
7. Once the downloads for `XGM2019e_2159` and `EGM96` are complete, you should have 4 files for each model (8 in total, in `.gdf` format). Save the `XGM2019e_2159` files in `sset.DIR_GEOG_DATUMS_XGM2019e_WGS84` and `EGM96` files in `sset.DIR_GEOG_DATUMS_EGM96_WGS84`.

### Global Mean Dynamic Ocean Topography (MDT) from AVISO+
**Note**: While this dataset has a relatively open license, you will first need to obtain a MY AVISO+ account, which requires verification from the AVISO+ team and may take several days or weeks.
1. Go to the following AVISO+ website for **MDT CNES-CLS18**: link [here](https://www.aviso.altimetry.fr/en/data/products/auxiliary-products/mdt/mdt-global-cnes-cls18.html).
2. Once on the page, download the dataset through your MY AVISO+ account (click on `access via MY AVISO+` link and follow the instructions).
3. After following the instructions, you will acquire the file `mdt_cnes_cls18_global.nc.gz`. Extract the file `mdt_cnes_cls18_global.nc` from the `.gz` file and save it as `sset.PATH_GEOG_MDT_RAW`.

### CIA World Factbook (compiled by Coleman [2020])

1. Travel to this [link](https://github.com/iancoleman/cia_world_factbook_api) (credit to Coleman [2020]), and scroll down to the `readme.md`.
2. In the **Data** section of the `readme.md` file, there should be a link on "Historical"; click on this link to travel to a `mega.nz` website having `weekly_json.7z` file.
3. After checking that the filename to download is `weekly_json.7z`, download the said file by clicking on the "Download" button.
4. When download is successful, import `weekly_json.7z` to the preferred directory (`sset.DIR_YPK_RAW` in this implementation).

### HydroSHEDS
1. Go to https://hydrosheds.org/downloads
2. Download the "standard" level-0 HydroBASINS files for each continent (use the Dropbox link if available--this appears as "NOTE: you may also download data from here." as of 8/16/21. Download the shapefiles into the directory defined in `sset.DIR_HYDROBASINS_RAW`

### Other SLIIDERS input datasets

There are three datasets that were manually constructed for use in `SLIIDERS`. They are available for download on Zenodo. Please download each file from the Zenodo deposit [here](https://doi.org/10.5281/zenodo.6450169) and copy to the paths designated for each dataset.

#### 1. `ne_coastline_lines_CIAM_wexp_or_gtsm.shp`
Path: `sset.PATH_CIAM_COASTLINES` (Download all files with the name `ne_coastline_lines_CIAM_wexp_or_gtsm` (but different extensions) to this directory.)  

Using the global coastlines derived from the Natural Earth layers, we included individual land masses formed by these coastlines only if they have either i) a non-zero value of exposure based on our exposure grid for population and capital assets, OR ii) if they have an associated coastal segment point, as derived primarily from the CoDEC GTSM station points. Association of a given land mass to nearby CoDEC point(s) was determined through manual inspection of the subset of land masses (n=636) with zero exposure in order to assess whether an intersecting or nearby station point represented that land area, resulting in the inclusion of 171 small land masses for which no population or capital is present but for which a coast point is associated.

#### 2. `gtsm_stations_eur_tothin.shp`
Path: `sset.DIR_GTSM_STATIONS_TOTHIN` (Download all files with the name `gtsm_stations_eur_tothin` (but different extensions) to this directory.)  

These 5,637 station points are a subset of the full CoDEC dataset (n=14,110) representing sites along European coastlines that are roughly five times more densely-spaced compared to the rest of the globe, as described in Muis et al. 2020. This subset of points are those that will be thinned by 5x to approximately match the density of CoDEC coast stations globally. Some manual inclusion criteria for this subset was applied in GIS due to the fact that simply seeking to select dense European stations based on the “station_name” field in the dataset, which contains the substring “eur” for all European locations, results in an over-selection of desired points (n=6,132), with many North African coastal points that are not densely-spaced containing this substring in their “station_name” as well. Therefore, European points were manually identified, with small islands, such as in the Mediterranean, included if their land mass contained 5 or more station points, which guarantees that they will be represented by at least one station point following the 5x thinning process. The resultant subset of points is used as a data input for the coastal segment construction in the preprocessing of the SLIIDERS dataset.

#### 3. `us_manual_protected_areas.parquet`
Path: `sset.PATH_US_MANUAL_PROTECTED_AREAS`  

The regions defined in this dataset represent a few areas in the United States that may have low-lying elevations, but are not vulnerable to flooding due to constructed barriers or since they are completely separated from the coastline by topographical features with much higher elevations. Areas protected by Louisiana levees were downloaded from the National Levee Database (https://levees.sec.usace.army.mil/), and areas corresponding to low-lying areas in California, Missouri, and Michigan that are not vulnerable to coastal flooding were created using spatial buffers around a central point.

### CoastalDEM v1.1
1. Acquire the global 1 arc-second CoastalDEM dataset from Climate Central (https://go.climatecentral.org/coastaldem/).
2. Save all 1-degree GeoTIFF files in `sset.DIR_COASTALDEM`